In [1]:
import os
os.chdir("..")
# Data structures
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
import pickle

# Necessary modules
from STRATEGY.BaseTradeEngine import BaseTradeEngine
from STRATEGY.Benchmark import Benchmark
from STRATEGY.Kalman import Kalman
from UTIL import tradeUtil as tu
import seaborn as sns

# Debug Tools
import sys
import importlib
import warnings
warnings.filterwarnings("ignore")

# Statistical tools
import statsmodels.tsa.stattools as ts

/home/fernandomlassop/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Read config
verbose = False

files = pd.DataFrame({'ODR': ['rya','ulvr','bpln','hsbcln'],
         'ADR': ['ryaay','ul','bp','hsbc']})

treatment1 = {'rm_outliers': False, 'rm_wide_spread': False, 'adjust_spread': False}

treatment2 = {'rm_outliers': True, 'rm_wide_spread': False, 'adjust_spread': False}

treatment3 = {'rm_outliers': True, 'rm_wide_spread': True, 'max_width': 10, 'adjust_spread': False}

treatment4 = {'rm_outliers': True, 'rm_wide_spread': True, 'max_width': 20, 'adjust_spread': False}

treatment5 = {'rm_outliers': True, 'rm_wide_spread': True, 'max_width': 20, 'adjust_spread': True, 'adjust_width': 10}

treatment6 = {'rm_outliers': True, 'rm_wide_spread': True, 'max_width': 20, 'adjust_spread': True, 'adjust_width': 7.5}

treatment7 = {'rm_outliers': False, 'rm_wide_spread': True, 'max_width': 20,'adjust_spread': True, 'adjust_width': 7.5}

treatments = [treatment1, treatment2, treatment3, treatment4, treatment5, treatment6, treatment7]

output = []
opt_settings = []

In [ ]:
default = {'entryZ': 2, 'exitZ': -1, 'lag': 0, 'resample': 5, 'rounding': 3, 'windowZ':120, 'windowOLS':150,
          'transaction_cost':0.0063}

values  =  {'entryZ': list(np.arange(0, 3.1, 0.5)), 'exitZ': [-5] + list(np.arange(-2.5, 0.6, 0.5)), 
           'lag': [0,1,2,5], 'resample': [1,5,10,20,60,250,500], 'rounding': [1,2,3], 
            'windowZ': list(np.arange(10, 71, 15)) + [100,150,300], 
            'windowOLS': list(np.arange(150, 1000, 150)) + [1500,2500]}

order   = np.array([['windowOLS','resample'],['windowOLS','windowZ'],['exitZ','entryZ'],
                    ['entryZ','resample'],  ['lag','rounding']])

for i, treat in enumerate(treatments):
    for index, row in files.iterrows():
        train_out, test_out, opt_set = tu.backtest_model(row['ODR'], row['ADR'], Benchmark, default, treat, 
                                             values, order, version = "Benchmark", clean_version = ("T" + str(i)),
                                                        train = [0,0.75], test = [0.75,1])
        output.extend(list((train_out,test_out)))
        opt_settings.append(opt_set)
    


-----------------------------------------------------------------------
rya vs ryaay pair trading using Benchmark and treatment T0
-----------------------------------------------------------------------
Observations in source  | ryaay : 110795 , rya : 134001 
After merging on date   | ryaay : 27152 , rya : 27152 
After applying filters  | ryaay : 27152 , rya : 27152 


100%|██████████| 12/12 [00:06<00:00,  1.80it/s]


-----------------------------------------------------------------------
ulvr vs ul pair trading using Benchmark and treatment T0
-----------------------------------------------------------------------
Observations in source  | ul : 157682 , ulvr : 134550 
After merging on date   | ul : 31631 , ulvr : 31631 
After applying filters  | ul : 31631 , ulvr : 31631 


 66%|██████▌   | 42/64 [00:44<00:23,  1.05s/it]

In [ ]:
default = {'lag': 0, 'resample': 5, 'rounding': 3,  'entryMult':2, 'exitMult':-3, 'transaction_cost': 0.0063}

values  =  {'lag': [0,1,2,5], 'resample': [1,5,10,20,60,250,500], 'rounding': [1,2,3], 
            'entryMult': list(np.arange(0.5, 5.1, 0.5)), 'exitMult': [-5] + list(np.arange(0.5, -3.1, -0.5))}


order   = np.array([['resample','entryMult'],['exitMult','entryMult'],['lag','rounding']])

for i, treat in enumerate(treatments):
    for index, row in files.iterrows():
        train_out, test_out, opt_set = tu.backtest_model(row['ODR'], row['ADR'], Kalman, default, treat, 
                                             values, order, version = "Kalman", clean_version = "T" +str(i),
                                                        train = [0,0.75], test = [0.75,1])
        output.extend(list((train_out,test_out)))
        opt_settings.append(opt_set)

In [ ]:
from IPython.display import display
results = pd.concat(output, axis=1)
results.columns = pd.MultiIndex.from_tuples(results.columns, names=['Data','Sample','Model','Treatment'])
display(results.transpose().sort_index(level = 'Data').style.background_gradient())

In [ ]:
idx = [o.columns[0] for o in output[::2]]
settings = pd.DataFrame(opt_settings,index=idx)
settings.index = pd.MultiIndex.from_tuples(settings.index, names=['Data','Sample','Model','Treatment'])


In [ ]:
results.transpose().to_pickle("./OUTPUT/results.pkl")
settings.to_pickle("./OUTPUT/opt_settings.pkl")

In [ ]:
#hi